In [0]:
from pyspark.sql.functions import concat, lit, col

# Truncate the target table to remove existing data
spark.sql("TRUNCATE TABLE platform_catalog.marketplace.image_urls")

# Generate 1000 image URLs using picsum.photos and create a DataFrame
df = spark.range(1, 1001).withColumn(
    "image_url",
    concat(
        lit("https://picsum.photos/id/"),
        col("id").cast("string"),
        lit("/800/600")
    )
).select("image_url")

# Append the generated image URLs to the target table
df.write.mode("append").saveAsTable("platform_catalog.marketplace.image_urls")

In [0]:
# Read all image URLs from the target table into a Spark DataFrame
df = spark.sql("SELECT * FROM platform_catalog.marketplace.image_urls")

# Display the DataFrame in a rich interactive format
display(df)

In [0]:
import requests

from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType

# Define a UDF to download an image from a URL and return its binary content
def download_image(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.content
    except Exception:
        return None

download_image_udf = udf(download_image, BinaryType())

# Load image URLs from the specified table
df = spark.table("platform_catalog.marketplace.image_urls")

# Apply the UDF to add a column containing the binary image data
df_with_binary = df.withColumn("image_binary", download_image_udf(df["image_url"]))

# Save the resulting DataFrame with image binaries to a new table
df_with_binary.write.mode("overwrite").saveAsTable("platform_catalog.marketplace.image_codes")

In [0]:
# Use ai_query with the binary column and save result as a new table
result_df = spark.sql("""
SELECT
  image_url,
  ai_query(
    'databricks-llama-4-maverick',
    'Descrivi dettagliatamente il contenuto di questa immagine.',
    files => image_binary
  ) AS descrizione
FROM platform_catalog.marketplace.image_codes
""")
result_df.write.mode("overwrite").saveAsTable("platform_catalog.marketplace.image_desc")

In [0]:
%sql
-- Selects all columns from the image_desc table in the marketplace schema
SELECT * 
FROM platform_catalog.marketplace.image_desc